# Get Data From Drive

In [1]:
!gdown "19FAA4_z7xzdqqtwnkLRL2oI4PuMWPYBZ&confirm=t"
!gdown "1ixS6ump2gFwwChlg4vrGyVa_9t0csLKd&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=19FAA4_z7xzdqqtwnkLRL2oI4PuMWPYBZ&confirm=t
To: /content/Dataset.zip
100% 845M/845M [00:09<00:00, 88.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ixS6ump2gFwwChlg4vrGyVa_9t0csLKd&confirm=t
To: /content/Patterns.zip
100% 271M/271M [00:02<00:00, 122MB/s] 


In [2]:
!unzip /content/Dataset.zip -d Dataset
!unzip /content/Patterns.zip -d Patterns

Archive:  /content/Dataset.zip
 extracting: Dataset/1644359422.816138.jpg  
  inflating: Dataset/1644359422.816138.json  
 extracting: Dataset/1644360063.82364.jpg  
  inflating: Dataset/1644360063.82364.json  
 extracting: Dataset/1644362482.7094185.jpg  
  inflating: Dataset/1644362482.7094185.json  
 extracting: Dataset/1644368971.6154966.jpg  
  inflating: Dataset/1644368971.6154966.json  
 extracting: Dataset/1644369207.922902.jpg  
  inflating: Dataset/1644369207.922902.json  
 extracting: Dataset/1644369658.7214494.jpg  
  inflating: Dataset/1644369658.7214494.json  
 extracting: Dataset/1644369774.9055796.jpg  
  inflating: Dataset/1644369774.9055796.json  
 extracting: Dataset/1644369801.9909956.jpg  
  inflating: Dataset/1644369801.9909956.json  
 extracting: Dataset/1644369842.0238864.jpg  
  inflating: Dataset/1644369842.0238864.json  
 extracting: Dataset/1644369905.2248352.jpg  
  inflating: Dataset/1644369905.2248352.json  
 extracting: Dataset/1644369972.1056488.jpg  
 

# Read image and pattern

In [3]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import json

In [4]:
img_name = "/content/Dataset/1644362482.7094185.jpg"
json_name = img_name[:-3] + "json"

In [ ]:
img = cv.imread(img_name)

f = open(json_name, encoding="utf8")
data = json.load(f)
f.close()

plt.figure(figsize=(20, 20))
plt.imshow(img[..., ::-1])

for shape in data["shapes"]: # array of json include taraks
  # shape :
  # {
  #           "label": "tarak",
  #           "points": [
  #               [
  #                   1239.6180419921875,
  #                   768.5914306640625
  #               ],
  #               [
  #                   1298.614990234375,
  #                   758.45849609375
  #               ],
  #               [
  #                   1300.58203125,
  #                   769.4680786132812
  #               ],
  #               [
  #                   1233.1680908203125,
  #                   777.0652465820312
  #               ]
  #           ],
  #           "group_id": null,
  #           "shape_type": "polygon",
  #           "flags": {}
  #       }
  points = np.array(shape['points'])
  ind = list(np.arange(points.shape[0])) + [0]
  plt.plot(points[ind, 0], points[ind, 1], 'r.-')
  # plt.plot(points[-1:-3:-1, 0], points[-1:-3:-1, 1], 'r')

In [ ]:
pattern = cv.imread('/content/Patterns/' + data['pattern']) # e.g. "AYLIN.tif"

plt.figure(figsize=(20, 20))
plt.imshow(pattern[..., ::-1])

# Prepare Sequence class to load & vectorize batches of data

In [5]:
def create_target(input_img_path):
  img = cv.imread(input_img_path)
  input_json_path = input_img_path[:-3] + 'json'
  f = open(input_json_path, encoding="utf8")
  data = json.load(f)
  f.close()

  shapes = data["shapes"]
  for shape in shapes:
    if shape["label"] == "tarak":
      points = np.array(shape["points"]) # taraks
      cv.fillPoly(img, pts=np.int32([points]), color=(0, 0, 255)) # Draw a filled polygon

  indices = np.where(img != (0, 0, 255))
  img[indices] = 0

  indices = np.where(img == (0, 0, 255))
  img[indices] = 128

  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  i1 = np.where(gray == gray[0, 0])
  i2 = np.where(gray != gray[0, 0])
  gray[i1] = 0 # not taraks
  gray[i2] = 128 # taraks

  return gray.copy()

In [ ]:
img_path = '/content/Dataset/1644360063.82364.jpg'
target = create_target(img_path)
plt.imshow(target, cmap="gray")

In [8]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
import os

class Tiles(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, is_pattern):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.is_pattern = is_pattern

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="uint8") # (100, 1600, 1600, 3)
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8") # (100, 1600, 1600, 1)
        pad = 70
        for j, path in enumerate(batch_input_img_paths):
            # input img
            img = load_img(path)
            img = np.array(img)
            img = img[pad:-pad, pad:-pad] # crop the paddings
            img = cv.resize(img, self.img_size, interpolation=cv.INTER_AREA) # resize
            x[j] = img

            # target img
            target_img = create_target(path) # our defined function
            target_img = target_img[pad:-pad, pad:-pad].copy()
            target_img = cv.resize(target_img, self.img_size, interpolation=cv.INTER_AREA)
            indices = np.where(target_img != 0)
            target_img[indices] = 1 # tarak (127 -> 1)
            y[j] = np.expand_dims(target_img, 2) # Expand the shape of an array. 1 -> [1] [ (1600,1600, ) -> (1600,1600,1) ]

        return x, y

# Prepare U-Net Xception-style model

In [9]:
# https://www.kaggle.com/code/meaninglesslives/unet-xception-keras-for-pneumothorax-segmentation/notebook
# https://www.researchgate.net/figure/U-Net-architecture-X-ception-style_fig4_364690133
# https://github.com/cnzakimuena/U-Net
# https://pyimagesearch.com/2022/02/21/u-net-image-segmentation-in-keras/
num_classes = 2 # tarak, not tarak
batch_size = 2
img_size = (1600, 1600)

In [ ]:
import math
import glob
from skimage import exposure
from skimage.exposure import match_histograms
from skimage import data
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
import keras
import random
import keras.backend as K
from keras.utils.vis_utils import plot_model

def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))


input = layers.Input((1600, 1600, 3))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(4, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(64, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(128, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(10000, activation="relu")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((1600, 1600, 3))
input_2 = layers.Input((1600, 1600, 3))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.

tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [ ]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'constrastive_loss' function with data ('margin') attached.
    """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing constrastive loss as floating point value.
        """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

def f1(y_true, y_pred):
  def recall(y_true, y_pred):
      """Recall metric.

      Only computes a batch-wise average of recall.

      Computes the recall, a metric for multi-label classification of
      how many relevant items are selected.
      """
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
      recall = true_positives / (possible_positives + K.epsilon())
      return recall

  def precision(y_true, y_pred):
      """Precision metric.

      Only computes a batch-wise average of precision.

      Computes the precision, a metric for multi-label classification of
      how many selected items are relevant.
      """
      true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
      predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
      precision = true_positives / (predicted_positives + K.epsilon())
      return precision
  precision = precision(y_true, y_pred)
  recall = recall(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))


epochs = 10
batch_size = 32
margin = 1
siamese.compile(loss=loss(margin=margin), optimizer="adam", metrics=["accuracy", f1])
siamese.summary()

In [ ]:
plot_model(siamese, show_shapes=True, show_layer_names=True)


In [ ]:
# # Free up RAM in case the model definition cells were run multiple times
# keras.backend.clear_session()

# # Build model
# model = get_model(img_size, num_classes)
# model.summary()
# keras.utils.plot_model(model)

# Read Prepared And Augmented Data Again

In [ ]:
import os

input_dir = "/content/Dataset"

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if not fname.endswith(".json")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path in input_img_paths[-15:]:
    print(input_path, "|", input_path[:-3] + 'json')

# Set aside a validation split

In [ ]:
import random

# Split our img paths into a training and a validation set
val_samples = 50
train_input_img_paths = input_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
# random.Random(0).shuffle(train_input_img_paths)
# random.Random(0).shuffle(val_input_img_paths)

# Instantiate data Sequences for each split
train_tile_gen = Tiles(batch_size, img_size, train_input_img_paths, False) #  """Helper to iterate over the data (as Numpy arrays)."""
train_pattern_gen = Tiles(batch_size, img_size, train_input_img_paths, True) #  """Helper to iterate over the data (as Numpy arrays)."""
val_gen = Tiles(batch_size, img_size, val_input_img_paths, False) #  """Helper to iterate over the data (as Numpy arrays)."""

## See Tile & Pattern & Mask

In [ ]:
plt.figure(figsize=(30, 30))
plt.subplot(2, 1, 1)
imgs, targets = train_gen[7]
plt.imshow(imgs[0]) # imgs[1]

plt.subplot(2, 1, 2)
plt.imshow(targets[0], cmap="gray") # targets[1]

In [ ]:
keras.backend.clear_session()

# Build model
model = get_model(img_size, num_classes)

In [ ]:
import tensorflow as tf
from keras.losses import binary_crossentropy

def dice_coef(y_true, y_pred, smooth=0.01):
  # ref: https://lars76.github.io/2018/09/27/loss-functions-for-segmentation.html#6
  # (batch_size, img1, img2, num_class)
  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.math.sigmoid(y_pred)
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred) # DC ≥ IoU

  return (numerator + smooth) / (denominator + smooth)

def dice_bce_loss(y_true, y_pred):
    """combine DICE and BCE"""
    # ref: https://www.kaggle.com/code/kmader/u-net-with-dice-and-augmentation/notebook
    dice_loss =  1 - dice_coef(y_true, y_pred)
    return 0.01 * binary_crossentropy(y_true, y_pred) + dice_loss

# Train the model

In [ ]:
from keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# save best weights
# ref: https://towardsdatascience.com/keras-callbacks-and-how-to-save-your-model-from-overtraining-244fc1de8608
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(
    filepath=filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min'
)

model.compile(optimizer=Adam(1e-4, decay=1e-6), loss=dice_bce_loss, metrics=[dice_coef])

epochs = 10
history = model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=[checkpoint])

In [ ]:
history = siamese.fit(
    [train1, train2],
    val_gen,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    shuffle=True
)

In [ ]:
# save latest weights
model.save_weights('unet.h5')

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model dice_coef')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#plot the training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('dice_bce_loss')
plt.savefig('model_training_history')
plt.show()

# Visualize predictions


In [ ]:
# load best weights
keras.backend.clear_session()
filepath = "my_best_model.epoch05-loss0.72.hdf5"
model = keras.models.load_model(filepath, custom_objects={"dice_bce_loss": dice_bce_loss, "dice_coef": dice_coef})

In [ ]:
from PIL import ImageOps

# Generate predictions for all images in the validation set
val_gen = Tiles(batch_size, img_size, val_input_img_paths) # 50 images (25 batches)
val_preds = model.predict(val_gen)

25/25 [==============================] - 33s 1s/step


In [ ]:
from IPython.display import Image, display

def get_visualization(i, img):
    """Quick utility to display a model's prediction."""
    probability = 1e-20
    predict = img.copy()
    mask = val_preds[i]
    mask = tf.math.sigmoid(mask)
    mask = np.array(mask)

    indices = np.where(mask >= probability)
    mask[indices] = 128

    indices = indices[0], indices[1]
    predict[indices] = (255, 0, 0)

    return mask, predict

# Display results for validation image #i
i = 0
batch = i // batch_size
j = i % batch_size
imgs, targets = val_gen[batch]

# Display input image
plt.figure(figsize=(30, 30))
plt.subplot(4, 1, 1)
plt.imshow(imgs[j])

# Display ground-truth target mask
plt.subplot(4, 1, 2)
plt.imshow(targets[j], cmap="gray")

# Display mask predicted by our model
plt.subplot(4, 1, 3)
mask, prediction = get_visualization(i, imgs[j])
plt.imshow(mask, cmap="gray")

plt.subplot(4, 1, 4)
plt.imshow(prediction)

In [ ]:
# define predict function

# define predict function
def my_get_visualization(img, mask):
    """Quick utility to display a model's prediction."""
    probability = 1e-20
    predict = img.copy()
    mask = tf.math.sigmoid(mask)
    mask = np.array(mask)

    indices = np.where(mask >= probability)
    mask[indices] = 128

    indices = indices[0], indices[1]
    predict[indices] = (255, 0, 0)

    return mask, predict


def predict_func(image, pattern):
    '''
    Receive the corresponding image and design and return the coordinates of the cracks inside it
    '''

    # load best weights
    keras.backend.clear_session()
    filepath = "my_best_model.epoch05-loss0.72.hdf5"
    model = keras.models.load_model(filepath, custom_objects={"dice_bce_loss": dice_bce_loss, "dice_coef": dice_coef})

    # load image
    img = Image.open(image)
    img = img.convert('RGB')
    img = np.array(img)
    img = img / 255.0
    img = img.astype(np.float32)

    # # load pattern
    # pattern = Image.open(pattern)
    # pattern = pattern.convert('RGB')
    # pattern = np.array(pattern)
    # pattern = pattern / 255.0
    # pattern = pattern.astype(np.float32)

    # predict
    # img = np.expand_dims(img, axis=0)
    # pattern = np.expand_dims(pattern, axis=0)
    # img = np.concatenate((img, pattern), axis=0)
    # img = np.expand_dims(img, axis=0)
    mask = model.predict(img)

    # method 1
    # mask, prediction = my_get_visualization(img[0, :, :, :], mask[0, :, :, :])
    # mask = np.where(mask >= 0.5, 1, 0)
    # mask = mask.astype(np.uint8)

    # method 2
    mask = tf.math.sigmoid(mask)
    mask = np.array(mask)
    mask = mask[0, :, :, 0]
    mask = np.where(mask >= 0.5, 1, 0)
    mask = mask.astype(np.uint8)

    # get coordinates
    contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    coordinates = []
    for i in range(len(contours)):
        x, y, w, h = cv.boundingRect(contours[i])
        coordinates.append([x, y, x+w, y+h])

    return coordinates
